In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_FRN.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CCY = 'TWD'
Type = 'FRN'
MTMDate = MktSheet.cell(1, 3).value
RW = MktSheet.cell(2, 8).value
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [4]:
ZeroSP = MktSheet.cell(43, 5).value
Zero6M = MktSheet.cell(45, 5).value
Zero1Y = MktSheet.cell(46, 5).value
Zero3Y = MktSheet.cell(48, 5).value
Zero5Y = MktSheet.cell(49, 5).value
Zero10Y = MktSheet.cell(50, 5).value
Zero30Y = MktSheet.cell(53, 5).value
print(ZeroSP)

0.004795999997252601


In [5]:
DateSP = evDate
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
Date3Y = Cal.advance(evDate, 3, ql.Years, BDC, False)
Date5Y = Cal.advance(evDate, 5, ql.Years, BDC, False)
Date10Y = Cal.advance(evDate, 10, ql.Years, BDC, False)
Date30Y = Cal.advance(evDate, 30, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [6]:
ListDate = list([DateSP, Date6M, Date1Y, Date3Y, Date5Y, Date10Y, Date30Y])
ListRate = list([ZeroSP, Zero6M, Zero1Y, Zero3Y, Zero5Y, Zero10Y, Zero30Y])
VecDate = ql.DateVector(ListDate)
VecRate = ql.DoubleVector(ListRate)
TSZero = ql.ZeroCurve(VecDate, VecRate, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)

In [7]:
FRNSheet = PstBook['FRN']
start_row = 2
end_row = 5
start_col = 0
end_col = 8
recordcount = end_col - start_col + 1
ListFRN = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in FRNSheet[i][start_col:end_col+1]]
    ListFRN.append(row)
    print(row)

['FRN001', 'TSMC', datetime.datetime(2021, 2, 21, 0, 0), datetime.datetime(2024, 2, 21, 0, 0), 12, 'Q', 0.02, 1000000, 0.018]
['FRN002', 'TSMC', datetime.datetime(2021, 4, 18, 0, 0), datetime.datetime(2028, 4, 18, 0, 0), 28, 'Q', 0.025, 1000000, 0.024]
['FRN003', 'CHT', datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2026, 6, 23, 0, 0), 20, 'Q', 0.018, 1000000, 0.02]
['FRN004', 'CHT', datetime.datetime(2021, 6, 1, 0, 0), datetime.datetime(2024, 6, 1, 0, 0), 12, 'Q', 0.016, 1000000, 0.016]


In [8]:
FRNBook = list()
for i in range(len(ListFRN)):
    row = ListFRN[i]
    record = list()
    record.append(row[0])
    record.append(row[1])
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8]) # CDS
    FRNBook.append(record)

In [9]:
FRNBook

[['FRN001',
  'TSMC',
  Date(21,2,2021),
  Date(21,2,2024),
  12,
  'Q',
  0.02,
  1000000,
  0.018],
 ['FRN002',
  'TSMC',
  Date(18,4,2021),
  Date(18,4,2028),
  28,
  'Q',
  0.025,
  1000000,
  0.024],
 ['FRN003',
  'CHT',
  Date(23,6,2021),
  Date(23,6,2026),
  20,
  'Q',
  0.018,
  1000000,
  0.02],
 ['FRN004',
  'CHT',
  Date(1,6,2021),
  Date(1,6,2024),
  12,
  'Q',
  0.016,
  1000000,
  0.016]]

In [10]:
h_TSZero = ql.YieldTermStructureHandle(TSZero)
FRNEngine = ql.DiscountingBondEngine(h_TSZero)

In [11]:
ListRate

[0.004795999997252601,
 0.005679705459626832,
 0.005055409226605943,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008870403457750699]

In [12]:
NoPoint = 7
NoCurve = NoPoint - 1
print(NoPoint)
ListRW = list([RW, RW, RW, RW, RW, RW])

7


In [13]:
P3M = ql.Period(3, ql.Months)
TWD = ql.TWDCurrency()
EOM = False
FixingDays = 2
TAIBOR3M = ql.IborIndex("TAIBOR", P3M, FixingDays, TWD, Cal, BDC, EOM, DC365, h_TSZero)

In [14]:
FixingDates = list()
FixingRates = list()
for i in range(33, 131):
    FixDate = DatetimeToDate(MktSheet.cell(i, 7).value)
    FixRate = MktSheet.cell(i, 8).value / 100.0
    FixingDates.append(FixDate)
    FixingRates.append(FixRate)
VecFixingDates = ql.DateVector(FixingDates)
VecFixingRates = ql.DoubleVector(FixingRates)    
TAIBOR3M.addFixings(VecFixingDates, VecFixingRates, True)

In [15]:
SenSheet = PstBook['CSR']
for i in range(0, len(FRNBook)):
    ListValue = list()

    record = FRNBook[i]
    issueDate = record[2]
    maturityDate = record[3]
    tenor = record[4]    
    spread = record[6]
    faceAmt = record[7]
    CDS = record[8]
    DGR = ql.DateGeneration.Backward
    sch = ql.Schedule(issueDate, maturityDate, P3M, Cal, BDC, BDC, DGR, False)    
    ListSpread = list()
    ListGearing = list()
    for j in range(tenor):
        ListSpread.append(spread)
        ListGearing.append(1.0)
    VecSpread = ql.DoubleVector(ListSpread)
    VecGearing = ql.DoubleVector(ListGearing)
    FRN = ql.FloatingRateBond(settlementDays, faceAmt, sch, TAIBOR3M, DC365, BDC, 2, 
                              VecGearing, VecSpread)
    ListCorpRate = list()
    for j in range(len(ListRate)):
        ListCorpRate.append(ListRate[j] + CDS)
    
    VecCorpRate = ql.DoubleVector(ListCorpRate)
    TSCorpZero = ql.ZeroCurve(VecDate, VecCorpRate, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
    h_TSCorpZero = ql.YieldTermStructureHandle(TSCorpZero)
    FRNEngine = ql.DiscountingBondEngine(h_TSCorpZero)
    FRN.setPricingEngine(FRNEngine)
    #Value = FRN.dirtyPrice() * faceAmt / 100.0
    Value = FRN.cleanPrice() * faceAmt / 100.0
    ListValue.append(Value)
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = record[1]
    SenSheet.cell(i+5, 3).value = ListValue[0]
        
    # Modify Curve begin here
    # ListZero = ListRate
    ListZero = ListCorpRate    
    VecDate = ql.DateVector(ListDate)    
    ListListZero = list()
    ListListZero.append(ListZero)
    for k in range(1, NoPoint-1):
        NewList = list()
        NewList.append(ListZero[0])
        for j in range(1, NoPoint):
            if(k==j):
                NewList.append(ListZero[j] + 0.0001)
            else:
                NewList.append(ListZero[j])
        ListListZero.append(NewList)    
    
    ListVecZero = list()
    ListTSZero = list()
    for k in range(NoCurve):
        AVecZero = ql.DoubleVector(ListListZero[k])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        ListTSZero.append(ATSZero)    
    
    CurvUpListListZero = list()
    CurvUpListListZero.append(ListZero)
    for k in range(1, NoCurve):
        NewList = list()
        NewList.append(ListZero[0])
        for s in range(1, NoPoint):
            if(k==s):
                NewList.append(ListZero[s] + RW)
            else:
                NewList.append(ListZero[s])
        CurvUpListListZero.append(NewList)    
    
    CurvDnListListZero = list()
    CurvDnListListZero.append(ListZero)
    for k in range(1, NoCurve):
        NewList = list()
        NewList.append(ListZero[0])
        for s in range(1, NoPoint):
            if(k==s):
                NewList.append(ListZero[s] - RW)
            else:
                NewList.append(ListZero[s])
        CurvDnListListZero.append(NewList)    
    # modify code end here
    
    for j in range(1, NoCurve):
        h_TS = ql.YieldTermStructureHandle(ListTSZero[j])
        FRNEngine = ql.DiscountingBondEngine(h_TS)
        FRN.setPricingEngine(FRNEngine)        
        #Value = FRN.dirtyPrice() * faceAmt / 100.0
        Value = FRN.cleanPrice() * faceAmt / 100.0
        ListValue.append(Value)
    
    ListSensitivity = list()
    for j in range(1, len(ListValue)):
        ListSensitivity.append((ListValue[j] - ListValue[0])*10000)
    
    ListWS = list()
    for j in range(0, len(ListSensitivity)):
        ListWS.append(ListRW[j] * ListSensitivity[j])
        
    for j in range(0, len(ListSensitivity)):    
        SenSheet.cell(i+5, j+5).value = ListWS[j]
    
    ListVecZero = list()
    CurvUpListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvUpListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvUpListTSZero.append(ATSZero)
                
    ListVecZero = list()
    CurvDnListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvDnListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvDnListTSZero.append(ATSZero)
        
    CurvUpListHTSZero = list()
    for j in range(NoCurve):
        CurvUpListHTSZero.append(ql.YieldTermStructureHandle(CurvUpListTSZero[j]))
        
    CurvDnListHTSZero = list()
    for j in range(NoCurve):
        CurvDnListHTSZero.append(ql.YieldTermStructureHandle(CurvDnListTSZero[j]))
        
    CurvUpValue = list()
    for j in range(NoCurve):
        FRNEngine = ql.DiscountingBondEngine(CurvUpListHTSZero[j])
        FRN.setPricingEngine(FRNEngine)
        #Value = FRN.dirtyPrice() * faceAmt / 100.0
        Value = FRN.cleanPrice() * faceAmt / 100.0
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoCurve):
        FRNEngine = ql.DiscountingBondEngine(CurvDnListHTSZero[j])
        FRN.setPricingEngine(FRNEngine)
        #Value = FRN.dirtyPrice() * faceAmt / 100.0
        Value = FRN.cleanPrice() * faceAmt / 100.0
        CurvDnValue.append(Value)        
        
    CurvatureUp = list()
    ShiftUpValue = list()
    DeltaChange = list()
    for j in range(1, NoCurve):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - RW * ListSensitivity[j-1]))
                
    for j in range(0, len(CurvatureUp)):    
        SenSheet.cell(i+5, j+12).value = CurvatureUp[j]
            
    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoCurve):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])    
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + RW * ListSensitivity[j-1]))

    for j in range(0, len(CurvatureDn)):    
        SenSheet.cell(i+5, j+19).value = CurvatureDn[j]
        
PstBook.save('Pst_FRN.xlsx')
PstBook.close()

In [16]:
ListValue

[999787.1998053389,
 999786.6727525122,
 999772.37208348,
 999518.5990498961,
 999787.1998053389,
 999787.1998053389]

In [17]:
CurvUpValue

[999787.1998053389,
 999682.1529142079,
 996831.3221322288,
 947530.5823852554,
 999787.1998053389,
 999787.1998053389]

In [18]:
CurvDnValue

[999787.1998053389,
 999892.979614094,
 1002762.6074361989,
 1055041.8147211066,
 999787.1998053389,
 999787.1998053389]